In [1]:
import os
import pyodbc
import numpy as np
import pandas as pd
import csv

In [2]:
conn = pyodbc.connect('Driver={SQL Server};' #Connections Removed
                      'Server={Server};'
                      'Database={Database};'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

In [4]:
seq = cursor.execute('''Select pitch.[GameId]
                        ,pitch.[EventNum]
                        ,pitch.[SequenceNumber]
                        ,[Pitch]
                        , pt.*
                        , maxseq
                        from stage.PitchByPitch pitch
                        left join work.ActionByCount abc
                        on pitch.GameId = abc.GameID
                        and pitch.EventNum = abc.Eventnum
                        and pitch.SequenceNumber  = abc.SequenceNumber
                        join stage.PitchType pt
                        on pitch.Pitch = pt.PitchCode
                        join (select Max(sequencenumber) maxseq, Gameid, eventnum from stage.PitchByPitch group by gameid, eventnum) maxs
                                                            on pitch.GameId = maxs.GameId
                                                            and pitch.EventNum = maxs.EventNum
                        where abc.Eventnum is null
                        Order by GameId, Eventnum, SequenceNumber
                            ''').fetchall()

In [7]:
for row in seq:
    gameid = row[0]
    event = row[1]
    action = row[2]
    pitchtype = row[3]
    isPitch = row[6]
    isStrike = row[7]
    isBall = row[8]
    foul = row[13]
    foulTip = row[14]
    maxseq = row[20]
    
    if action == 1:
        strikes = 0
        balls = 0
    
    currentBalls = balls
    currentStrikes = strikes
    
    if isBall == 1:
        balls += 1
    
    if isStrike == 1:
        strikes += 1
    
    if foul == 1 and strikes > 2:
        strikes -= 1
    
    if pitchtype == 'L' and currentStrikes == 2:
        strikes += 1
    
    if foulTip == 1 and currentStrikes == 2 and pitchnum == maxseq:
        strikes += 1
    
    lines = [gameid, event, action, pitchtype, currentBalls, currentStrikes, None, None, None, None
            , balls, strikes]
    cmd = """Insert into work.ActionByCount 
            Values ('{}', {}, {}, '{}', {}, {}, NULL, NULL, NULL, NULL, {},{})
    """.format(gameid, event, action, pitchtype, currentBalls, currentStrikes, balls, strikes)
    
    cursor.execute(cmd)
    
    print('executed: {}'.format(cmd))

executed: Insert into work.ActionByCount 
            Values ('ANA201208100', 72, 1, '.', 0, 0, NULL, NULL, NULL, NULL, 0,0)
    
executed: Insert into work.ActionByCount 
            Values ('ANA201208100', 72, 2, 'S', 0, 0, NULL, NULL, NULL, NULL, 0,1)
    
executed: Insert into work.ActionByCount 
            Values ('ANA201208100', 72, 3, 'S', 0, 1, NULL, NULL, NULL, NULL, 0,2)
    
executed: Insert into work.ActionByCount 
            Values ('ANA201208100', 72, 4, '*', 0, 2, NULL, NULL, NULL, NULL, 0,2)
    
executed: Insert into work.ActionByCount 
            Values ('ANA201208100', 72, 5, 'B', 0, 2, NULL, NULL, NULL, NULL, 1,2)
    
executed: Insert into work.ActionByCount 
            Values ('ANA201208100', 72, 6, 'B', 1, 2, NULL, NULL, NULL, NULL, 2,2)
    
executed: Insert into work.ActionByCount 
            Values ('ANA201506280', 80, 1, 'C', 0, 0, NULL, NULL, NULL, NULL, 0,1)
    
executed: Insert into work.ActionByCount 
            Values ('ANA201506280', 80, 2, 'S', 

In [8]:
cursor.commit()